In [1]:
import time
print(time.ctime())

Sun Apr  3 22:34:16 2022


In [2]:
import os
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, SGD

Init Plugin
Init Graph Optimizer
Init Kernel


In [4]:
#some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 2000
LATENT_DIM = 25

In [5]:
# load in the data
input_texts = []
target_texts = []
for line in open('robert_frost.txt'):
    line = line.strip()
    if not line:
        continue
    
    input_line = '<sos> '+line
    target_line = line+' <eos>'
    
    input_texts.append(input_line)
    target_texts.append(target_line)
    
all_lines = input_texts + target_texts

#convert the sentences (string) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
print(input_sequences[0])
print(target_sequences[0])

[1, 104, 537, 538, 9, 7, 539, 540]
[104, 537, 538, 9, 7, 539, 540, 2]


In [6]:
#find the max sequence length
max_sequence_length_from_data = max(len(s) for s in input_sequences)
print("Maximum seq length from the input_sequences:", max_sequence_length_from_data)

Maximum seq length from the input_sequences: 12


In [7]:
#get word -> integer mapping
word2idx = tokenizer.word_index
print(f"Found {len(word2idx)} unique tokens")

Found 3056 unique tokens


In [8]:
assert('<sos>' in word2idx)
assert('<eos>' in word2idx)

In [9]:
#pad sequence so that we get NxT matrix
max_sequence_length = min(max_sequence_length_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding="post")
target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding="post")
print('Shape of data tensor: ', input_sequences.shape)
print(type(input_sequences))
print(input_sequences)

Shape of data tensor:  (1436, 12)
<class 'numpy.ndarray'>
[[  1 104 537 ...   0   0   0]
 [  1   5 541 ...   0   0   0]
 [  1   5  27 ...   0   0   0]
 ...
 [  1  53 564 ...   0   0   0]
 [  1  69 438 ... 733   0   0]
 [  1   4  75 ...   0   0   0]]


In [10]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('../../Natural Language Processing with Deep Learning in Python/2. Beginner\'s Corner Working with Word Vectors/glove.6B/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
    # is just a space-separated text file in the format:
    # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 400000 word vectors.


In [11]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all zeros.
          embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [12]:
#one-hot the target (can't use sparse entropy when the targets are sequences)
one_hot_targets = np.zeros((len(target_sequences), max_sequence_length, num_words)) #mind the order of dimensions
for i, target_sequence in enumerate(target_sequences):
    for t, word in enumerate(target_sequence):
        if word>0:
            one_hot_targets[i, t, word] = 1

In [13]:
# load pre-trained word embeddings into an Embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  # trainable=False
)

In [14]:
print('Building model...')

#create an LSTM network with a single LSTM
input_ = Input(shape=(max_sequence_length))
initial_h = Input(shape=(LATENT_DIM, ))
initial_c = Input(shape=(LATENT_DIM, ))
x = embedding_layer(input_)
lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, _, _ = lstm(x, initial_state=[initial_h, initial_c]) #don't need the states here
dense = Dense(num_words, activation='softmax')
output = dense(x)

model = Model([input_, initial_h, initial_c], output)
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  #optimizer=opt,
  # optimizer=SGD(lr=0.01, momentum=0.9),
  metrics=['accuracy']
)

Building model...
Metal device set to: Apple M1


2022-04-03 22:34:20.972831: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-03 22:34:20.972921: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
print('Training model...')
z = np.zeros((len(input_sequences), LATENT_DIM))
r = model.fit(
  [input_sequences, z, z],
  one_hot_targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

Training model...
Epoch 1/2000


2022-04-03 22:34:21.200049: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-03 22:34:21.200223: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-03 22:34:28.483715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-03 22:34:28.691003: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-03 22:34:29.769512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 10s 82ms/step - loss: 5.5887 - accuracy: 0.0108 - val_loss: 5.5991 - val_accuracy: 0.0223
Epoch 2/2000
1/9 [==>...........................] - ETA: 0s - loss: 5.3693 - accuracy: 0.0312

2022-04-03 22:34:30.781180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-03 22:34:30.815551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 39ms/step - loss: 5.4590 - accuracy: 0.0325 - val_loss: 5.4633 - val_accuracy: 0.0226
Epoch 3/2000
9/9 [==============================] - 0s 38ms/step - loss: 5.3369 - accuracy: 0.0326 - val_loss: 5.3352 - val_accuracy: 0.0226
Epoch 4/2000
9/9 [==============================] - 0s 38ms/step - loss: 5.2296 - accuracy: 0.0335 - val_loss: 5.2216 - val_accuracy: 0.0226
Epoch 5/2000
9/9 [==============================] - 0s 38ms/step - loss: 5.0541 - accuracy: 0.0330 - val_loss: 5.1217 - val_accuracy: 0.0226
Epoch 6/2000
9/9 [==============================] - 0s 39ms/step - loss: 4.9279 - accuracy: 0.0334 - val_loss: 5.0354 - val_accuracy: 0.0226
Epoch 7/2000
9/9 [==============================] - 0s 40ms/step - loss: 4.8150 - accuracy: 0.0328 - val_loss: 4.9618 - val_accuracy: 0.0226
Epoch 8/2000
9/9 [==============================] - 0s 37ms/step - loss: 4.7434 - accuracy: 0.0335 - val_loss: 4.8999 - val_accuracy: 0.0226
Epoch 9/2000
9/9 [========

...

9/9 [==============================] - 0s 41ms/step - loss: 3.9606 - accuracy: 0.1061 - val_loss: 5.3449 - val_accuracy: 0.0877
Epoch 346/2000
9/9 [==============================] - 0s 42ms/step - loss: 3.8920 - accuracy: 0.1054 - val_loss: 5.3441 - val_accuracy: 0.0877
Epoch 347/2000
9/9 [==============================] - 0s 41ms/step - loss: 3.9705 - accuracy: 0.1056 - val_loss: 5.3443 - val_accuracy: 0.0880
Epoch 348/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.9303 - accuracy: 0.1067 - val_loss: 5.3441 - val_accuracy: 0.0877
Epoch 349/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.9464 - accuracy: 0.1077 - val_loss: 5.3435 - val_accuracy: 0.0877
Epoch 350/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.9218 - accuracy: 0.1064 - val_loss: 5.3432 - val_accuracy: 0.0874
Epoch 351/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.9369 - accuracy: 0.1068 - val_loss: 5.3432 - val_accuracy: 0.0877
Epoch 352/2000

9/9 [==============================] - 0s 40ms/step - loss: 3.8777 - accuracy: 0.1098 - val_loss: 5.3415 - val_accuracy: 0.0906
Epoch 403/2000
9/9 [==============================] - 0s 41ms/step - loss: 3.8115 - accuracy: 0.1067 - val_loss: 5.3408 - val_accuracy: 0.0903
Epoch 404/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.8607 - accuracy: 0.1085 - val_loss: 5.3407 - val_accuracy: 0.0906
Epoch 405/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.8917 - accuracy: 0.1072 - val_loss: 5.3408 - val_accuracy: 0.0906
Epoch 406/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.8930 - accuracy: 0.1088 - val_loss: 5.3411 - val_accuracy: 0.0911
Epoch 407/2000
9/9 [==============================] - 0s 37ms/step - loss: 3.8577 - accuracy: 0.1062 - val_loss: 5.3413 - val_accuracy: 0.0909
Epoch 408/2000
9/9 [==============================] - 0s 40ms/step - loss: 3.8749 - accuracy: 0.1078 - val_loss: 5.3414 - val_accuracy: 0.0911
Epoch 409/2000

9/9 [==============================] - 0s 39ms/step - loss: 3.7665 - accuracy: 0.1085 - val_loss: 5.3485 - val_accuracy: 0.0920
Epoch 460/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.7408 - accuracy: 0.1100 - val_loss: 5.3491 - val_accuracy: 0.0923
Epoch 461/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.8026 - accuracy: 0.1085 - val_loss: 5.3500 - val_accuracy: 0.0920
Epoch 462/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.7504 - accuracy: 0.1107 - val_loss: 5.3504 - val_accuracy: 0.0917
Epoch 463/2000
9/9 [==============================] - 0s 38ms/step - loss: 3.8188 - accuracy: 0.1096 - val_loss: 5.3516 - val_accuracy: 0.0920
Epoch 464/2000
9/9 [==============================] - 0s 37ms/step - loss: 3.8166 - accuracy: 0.1092 - val_loss: 5.3524 - val_accuracy: 0.0920
Epoch 465/2000
9/9 [==============================] - 0s 38ms/step - loss: 3.7784 - accuracy: 0.1080 - val_loss: 5.3516 - val_accuracy: 0.0920
Epoch 466/2000

9/9 [==============================] - 0s 40ms/step - loss: 3.6887 - accuracy: 0.1108 - val_loss: 5.3726 - val_accuracy: 0.0903
Epoch 517/2000
9/9 [==============================] - 0s 38ms/step - loss: 3.7462 - accuracy: 0.1119 - val_loss: 5.3732 - val_accuracy: 0.0903
Epoch 518/2000
9/9 [==============================] - 0s 38ms/step - loss: 3.7382 - accuracy: 0.1106 - val_loss: 5.3745 - val_accuracy: 0.0894
Epoch 519/2000
9/9 [==============================] - 0s 41ms/step - loss: 3.7202 - accuracy: 0.1113 - val_loss: 5.3747 - val_accuracy: 0.0900
Epoch 520/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.7071 - accuracy: 0.1110 - val_loss: 5.3746 - val_accuracy: 0.0903
Epoch 521/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.7077 - accuracy: 0.1117 - val_loss: 5.3749 - val_accuracy: 0.0900
Epoch 522/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.7213 - accuracy: 0.1107 - val_loss: 5.3762 - val_accuracy: 0.0906
Epoch 523/2000

9/9 [==============================] - 0s 40ms/step - loss: 3.6943 - accuracy: 0.1124 - val_loss: 5.3963 - val_accuracy: 0.0903
Epoch 574/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6393 - accuracy: 0.1124 - val_loss: 5.3966 - val_accuracy: 0.0900
Epoch 575/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6497 - accuracy: 0.1149 - val_loss: 5.3962 - val_accuracy: 0.0906
Epoch 576/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6736 - accuracy: 0.1148 - val_loss: 5.3954 - val_accuracy: 0.0900
Epoch 577/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6320 - accuracy: 0.1137 - val_loss: 5.3949 - val_accuracy: 0.0903
Epoch 578/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6633 - accuracy: 0.1157 - val_loss: 5.3981 - val_accuracy: 0.0900
Epoch 579/2000
9/9 [==============================] - 0s 39ms/step - loss: 3.6585 - accuracy: 0.1129 - val_loss: 5.3975 - val_accuracy: 0.0900
Epoch 580/2000

In [ ]:
print('Training model...')
z = np.zeros((len(input_sequences), LATENT_DIM))
r = model.fit(
  [input_sequences, z, z],
  one_hot_targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
# make a sampling model
input2 = Input(shape=(1,)) # we'll only input one word at a time
x = embedding_layer(input2)
x, h, c = lstm(x, initial_state=[initial_h, initial_c]) # now we need states to feed back in
output2 = dense(x)
sampling_model = Model([input2, initial_h, initial_c], [output2, h, c])

In [ ]:
# reverse word2idx dictionary to get back words
# during prediction
idx2word = {v:k for k, v in word2idx.items()}

In [ ]:
def sample_line():
  # initial inputs
  np_input = np.array([[ word2idx['<sos>'] ]])
  h = np.zeros((1, LATENT_DIM))
  c = np.zeros((1, LATENT_DIM))

  # so we know when to quit
  eos = word2idx['<eos>']

  # store the output here
  output_sentence = []

  for _ in range(max_sequence_length):
    o, h, c = sampling_model.predict([np_input, h, c])

    # print("o.shape:", o.shape, o[0,0,:10])
    # idx = np.argmax(o[0,0])
    probs = o[0,0]
    if np.argmax(probs) == 0:
      print("wtf")
    probs[0] = 0
    probs /= probs.sum()
    idx = np.random.choice(len(probs), p=probs)
    if idx == eos:
      break

    # accuulate output
    output_sentence.append(idx2word.get(idx, '<WTF %s>' % idx))

    # make the next input into model
    np_input[0,0] = idx

  return ' '.join(output_sentence)

# generate a 4 line poem
while True:
  for _ in range(4):
    print(sample_line())

  ans = input("---generate another? [Y/n]---")
  if ans and ans[0].lower().startswith('n'):
    break